# Sam’s Subs ELT Project: Assignment Deliverable
## Group 6: Adrian Chavez-Loya, Hunter Lovell, Wayne Montgomery

## Project Overview

### Objective
Sam’s Subs, a popular sandwich chain, aims to optimize its data infrastructure by developing a data warehouse for analyzing its operations and customer behavior. In this assignment, we will implement an Extract, Load, and Transform (ELT) process. This will include extracting data from the SQL Server database, loading it into Snowflake, and transforming it into a star schema using dbt.

### Assignment Requirements

1. **Extract and Load (Airbyte):**  
   Set up Airbyte to extract data from SQL Server and load it into the Snowflake `RAW_SAMSSUBS` schema.
   
2. **Transform (dbt):**  
   Using dbt, transform raw data from the `RAW_SAMSSUBS` schema and create dimensional and fact tables in the `SAMS_SUBS` schema in Snowflake.
   
3. **Deliverables:**
   - A Jupyter Notebook summarizing the process with SQL and YAML code blocks used to with dbt to transform data and create data warehouse structure.

---

## Database Structure and Schema Design

The following tables are created to populate the data warehouse. In addition to tables populated with raw data, there will be tables created for entities where data is not yet available. These tables will be included in the dimensional model to support future data loading.

### Dimensions
- **Customer Dimension** (`customer_dim`): Stores customer details like name and contact information.
- **Date Dimension** (`date_dim`): Stores date details for analysis by day, month, quarter, and year.
- **Employee Dimension** (`employee_dim`): Stores employee information.
- **Product Dimension** (`product_dim`): Stores details about products.
- **Store Dimension** (`store_dim`): Stores store location details.

### Fact Tables
- **Order Fact** (`order_fact`): Stores order information with references to customers, employees, stores, and order details.
- **Order Line Fact** (`order_line_fact`): Stores individual line items within orders, including product and pricing details.

### Placeholder Table
- **Inventory** (`inventory_dim`): A placeholder table to store inventory data, which is currently unavailable. This table will support future data loading.

---


## 1. Configure `profiles.yml`

The `profiles.yml` file for dbt should be configured as follows. It connects to Snowflake, with the target database `GROUP6PROJECT`, and transforms data into the `SAMS_SUBS` schema.

### `profiles.yml`

```yaml
group6_project:
  target: dev
  outputs:
    dev:
      type: snowflake
      account: sfedu02-etb90388
      user: AnimalName ## Replace User
      password: [YourPasswordHere]  # Replace password!
      role: TRAINING_ROLE
      database: GROUP6PROJECT
      warehouse: GROUP6_WH
      schema: SAMS_SUBS
      threads: 4
      client_session_keep_alive: False


## 2. Define Sources in dbt

The `_src_sams_subs.yml` file defines the source tables for dbt to access from the `RAW_SAMSSUBS` schema. This file should be saved inside the `models/sams_subs` folder.

### `_src_sams_subs.yml`

```yaml
version: 2

sources:
  - name: raw_data
    schema: RAW_SAMSSUBS
    tables:
      - name: Customer
      - name: Employee
      - name: Order
      - name: OrderLineItem
      - name: Product
      - name: Sandwich
      - name: Store

---

## 3. Transform Raw Data into Dimension and Fact Tables

These SQL files create the necessary dimension and fact tables in the `SAMS_SUBS` schema by transforming the raw data from the `RAW_SAMSSUBS` schema.

### `customer_dim.sql`
This file creates the `customer_dim` table in the `SAMS_SUBS` schema.

In [ ]:
{{ config(
    materialized='table',
    schema='SAMS_SUBS',
    name='customer_dim'
) }}

with raw_data as (
    select
        CustomerID as customer_id,
        FirstName as first_name,
        LastName as last_name,
        DOB as dob,
        PhoneNumber as phone_number
    from {{ source('raw_data', 'Customer') }}
)

select 
    row_number() over () as cust_key,
    customer_id,
    first_name,
    last_name,
    dob,
    phone_number
from raw_data;

---

### `date_dim.sql`
The `date_dim.sql` file creates the `date_dim` table in the `SAMS_SUBS` schema by extracting date-related details from the `Order` table.


In [ ]:
{{ config(
    materialized='table',
    schema='SAMS_SUBS',
    name='date_dim'
) }}

with raw_data as (
    select
        distinct OrderDate as date_id,
        DAYOFWEEK(OrderDate) as day_of_week,
        EXTRACT(MONTH from OrderDate) as month,
        EXTRACT(QUARTER from OrderDate) as quarter,
        EXTRACT(YEAR from OrderDate) as year
    from {{ source('raw_data', 'Order') }}
)

select 
    row_number() over () as date_key,
    date_id,
    day_of_week,
    month,
    quarter,
    year
from raw_data;

---

### `employee_dim.sql`
The `employee_dim.sql` file creates the `employee_dim` table in the `SAMS_SUBS` schema to store information about employees.

In [ ]:
{{ config(
    materialized='table',
    schema='SAMS_SUBS',
    name='employee_dim'
) }}

with raw_data as (
    select
        EmployeeID as employee_id,
        FirstName as first_name,
        LastName as last_name,
        DOB as dob
    from {{ source('raw_data', 'Employee') }}
)

select 
    row_number() over () as employee_key,
    employee_id,
    first_name,
    last_name,
    dob
from raw_data;

---

### `product_dim.sql`
The `product_dim.sql` file creates the `product_dim` table in the `SAMS_SUBS` schema to store details about products.

In [ ]:
{{ config(
    materialized='table',
    schema='SAMS_SUBS',
    name='product_dim'
) }}

with raw_data as (
    select
        ProductID as product_id,
        [Type] as product_type,
        Name as product_name,
        Cost as cost,
        Calories as calories
    from {{ source('raw_data', 'Product') }}
)

select 
    row_number() over () as product_key,
    product_id,
    product_type,
    product_name,
    NULL as length,
    cost,
    calories,
    NULL as description
from raw_data;


---

### `store_dim.sql`
The `store_dim.sql` file creates the `store_dim` table in the `SAMS_SUBS` schema to store location details about each store.

In [ ]:
{{ config(
    materialized='table',
    schema='SAMS_SUBS',
    name='store_dim'
) }}

with raw_data as (
    select
        StoreID as store_id,
        Address as address,
        City as city,
        State as state,
        Zipcode as zipcode
    from {{ source('raw_data', 'Store') }}
)

select 
    row_number() over () as store_key,
    store_id,
    address,
    city,
    state,
    zipcode
from raw_data;

---

### `order_fact.sql`
The `order_fact.sql` file creates the `order_fact` table in the `SAMS_SUBS` schema, transforming order data into a fact table.


In [ ]:
{{ config(
    materialized='table',
    schema='SAMS_SUBS',
    name='order_fact'
) }}

with raw_data as (
    select
        OrderNo as order_no,
        EmployeeID as employee_id,
        CustomerID as customer_id,
        StoreID as store_id,
        TotalPrice as total_price,
        OrderDate as order_date,
        OrderMethod as order_method
    from {{ source('raw_data', 'Order') }}
)

select 
    row_number() over () as order_key,
    order_no,
    employee_id as employee_key,
    customer_id as cust_key,
    store_id as store_key,
    total_price,
    order_date,
    order_method
from raw_data;

---

### `order_line_fact.sql`
The `order_line_fact.sql` file creates the `order_line_fact` table in the `SAMS_SUBS` schema to store individual line items within orders.

In [ ]:
{{ config(
    materialized='table',
    schema='SAMS_SUBS',
    name='order_line_fact'
) }}

with raw_data as (
    select
        LineID as line_id,
        OrderNo as order_no,
        ProductID as product_id,
        Quantity as quantity,
        LinePrice as line_price
    from {{ source('raw_data', 'OrderLineItem') }}
)

select 
    row_number() over () as order_line_key,
    order_no,
    product_id as product_key,
    quantity,
    line_price
from raw_data;

---